# Making the dataset 

Data chose from the Kaggle dataset [Toursim](https://www.kaggle.com/datasets/imtkaggleteam/tourism?select=1-+international-tourist-arrivals-by-region-of-origin.csv)

Uploading the chosen csvs.

In [5]:
import pandas as pd

In [6]:
# related industry workers
tourism_workers = pd.read_csv(r"..\Data\2- number-of-individuals-employed-in-tourism-related-industries-per-1000-people.csv")
food_bev_workers = pd.read_csv(r"..\Data\11- number-of-people-employed-in-food-and-beverage-serving-activities-per-1000-population.csv")

# economics data
gdp_tourism_proportion = pd.read_csv(r"..\Data\7- tourism-gdp-proportion-of-total-gdp.csv")
business_to_tourists_ratio = pd.read_csv(r"..\Data\35- ratio-of-business-trips-to-trips-for-personal-reasons.csv")


# stay related data
avg_len_stay = pd.read_csv(r"..\Data\10- average-length-of-stay.csv")
inbound_outbound_ratio = pd.read_csv(r"..\Data\36- ratio-of-inbound-to-outbound-tourists.csv")
inbound_arrivals = pd.read_csv(r"..\Data\23- international-tourist-trips-per-1000-people.csv")
domestic_tourists = pd.read_csv(r"..\Data\38- domestic-trips-by-tourists-per-1000-people.csv")

Looking at each data separately and building a feature that incorporates data from all the years for that specific country.

## Related Industry workers

In [7]:
tourism_workers.rename(columns={"Entity":"Country", "Employment (total) per 1000 people":"tourism_employment_per_1000"}, inplace=True)
tourism_workers.head(5)


,Country,Code,Year,tourism_employment_per_1000
0,Albania,ALB,2004,4.897753
1,Albania,ALB,2005,5.144040
2,Albania,ALB,2006,5.560381
3,Albania,ALB,2007,6.182611
4,Albania,ALB,2008,5.928807


Given that we want to have only one value for a certain country, we can apply exponential weighted average on the yearly data we have, thus taking into account historical values while giving more emphasis to current data. 

In [8]:
tourism_workers = tourism_workers.sort_values(['Country', 'Year'])

# Apply exponentially weighted mean within each country
tourism_workers['tourism_employment_per_1000_over_time'] = (
    tourism_workers.groupby('Country')['tourism_employment_per_1000']
    .transform(lambda x: x.ewm(com=0.5, adjust=True).mean())
)

# Keep only the latest year data for each country
tourism_workers_adjusted = tourism_workers.loc[tourism_workers.groupby('Country')['Year'].idxmax()]

How the early features is transformed based on the yearly data:

In [9]:
tourism_workers.query("Country == 'Albania'")


,Country,Code,Year,tourism_employment_per_1000,tourism_employment_per_1000_over_time
0,Albania,ALB,2004,4.897753,4.897753
1,Albania,ALB,2005,5.144040,5.082468
2,Albania,ALB,2006,5.560381,5.413331
3,Albania,ALB,2007,6.182611,5.932595
4,Albania,ALB,2008,5.928807,5.930059
5,Albania,ALB,2009,6.381048,6.231131
6,Albania,ALB,2010,7.551310,7.111653
7,Albania,ALB,2011,7.825805,7.587827
8,Albania,ALB,2012,7.399235,7.462093
9,Albania,ALB,2013,8.763365,8.329622


In [10]:
tourism_workers_adjusted.reset_index(drop=True, inplace=True)
tourism_workers_adjusted.drop(columns=["tourism_employment_per_1000", "Year"], inplace=True)
tourism_workers_adjusted.drop_duplicates(inplace=True)
tourism_workers_adjusted


,Country,Code,tourism_employment_per_1000_over_time
0,Albania,ALB,15.608840
1,Algeria,DZA,7.411779
2,Angola,AGO,2.439200
3,Argentina,ARG,28.270541
4,Australia,AUS,21.464763
...,...,...,...
105,United States,USA,13.763542
106,United States Virgin Islands,VIR,58.525202
107,Uruguay,URY,35.711428
108,Uzbekistan,UZB,1.545283


Let's do the same for food and beverages workers.

In [11]:
food_bev_workers.rename(columns={"Entity":"Country", "Employment (food and beverage serving activities) per 1000 people":"food_employment_per_1000"}, inplace=True)
food_bev_workers.head(5)

,Country,Code,Year,food_employment_per_1000
0,Angola,AGO,2007,1.708873
1,Angola,AGO,2008,2.238432
2,Angola,AGO,2009,2.413088
3,Angola,AGO,2010,2.393705
4,Angola,AGO,2011,2.526555


In [12]:
food_bev_workers = food_bev_workers.sort_values(['Country', 'Year'])

# Apply exponentially weighted mean within each country
food_bev_workers['food_employment_per_1000_over_time'] = (
    food_bev_workers.groupby('Country')['food_employment_per_1000']
    .transform(lambda x: x.ewm(com=0.5, adjust=True).mean())
)

# Keep only the latest year data for each country
food_bev_workers_adjusted = food_bev_workers.loc[food_bev_workers.groupby('Country')['Year'].idxmax()]

In [13]:
food_bev_workers_adjusted.reset_index(drop=True, inplace=True)
food_bev_workers_adjusted.drop(columns=["food_employment_per_1000", "Year"], inplace=True)
food_bev_workers_adjusted.drop_duplicates(inplace=True)
food_bev_workers_adjusted

,Country,Code,food_employment_per_1000_over_time
0,Angola,AGO,0.710283
1,Argentina,ARG,11.444930
2,Australia,AUS,7.825559
3,Austria,AUT,11.464300
4,Bahrain,BHR,31.267619
...,...,...,...
80,United Arab Emirates,ARE,22.769673
81,United Kingdom,GBR,21.678139
82,United States,USA,3.456045
83,United States Virgin Islands,VIR,25.238442


Making a function to simplify the process. 

In [14]:
def apply_ewm_and_filter(df, column_name, new_column_name='EWMA_Result', com_value=0.5):
    """
    Apply exponentially weighted mean (EWM) on the specified column for each country,
    keep only the latest year data for each country, and clean the dataframe.

    Parameters:
    - df (DataFrame): The input DataFrame.
    - column_name (str): The column name on which EWM is applied.
    - new_column_name (str): The name of the new column where EWM results are stored (default is 'EWMA_Result').
    - com_value (float): The center of mass (com) value for the EWM (default is 0.5).

    Returns:
    - DataFrame: A cleaned DataFrame with only the latest data and the calculated EWM.
    """
    
    # Step 1: Sort by 'Country' and 'Year'
    df = df.sort_values(['Country', 'Year'])
    
    # Step 2: Apply exponentially weighted mean within each country
    df[new_column_name] = (
        df.groupby('Country')[column_name]
        .transform(lambda x: x.ewm(com=com_value, adjust=True).mean())
    )
    
    # Step 3: Keep only the latest year data for each country
    df_latest = df.loc[df.groupby('Country')['Year'].idxmax()]
    
    # Step 4: Reset index and drop unnecessary columns
    df_latest.reset_index(drop=True, inplace=True)
    df_latest.drop(columns=[column_name, 'Year'], inplace=True)
    
    # Step 5: Drop duplicates if any
    df_latest.drop_duplicates(inplace=True)
    
    return df_latest



## Economics Data

In [15]:
gdp_tourism_proportion.rename(columns={"Entity":"Country", "Tourism GDP as a proportion of Total":"tourism_gdp_percentage"}, inplace=True)
gdp_tourism_proportion.head()

,Country,Code,Year,tourism_gdp_percentage
0,Algeria,DZA,2008,0.8
1,Algeria,DZA,2009,1.1
2,Algeria,DZA,2010,1.0
3,Algeria,DZA,2011,0.9
4,Algeria,DZA,2012,0.9


In [16]:
business_to_tourists_ratio.rename(columns={"Entity":"Country", "Business/Personal Ratio of Purpose of Travel":"business_to_personal_ratio"}, inplace=True)
business_to_tourists_ratio.head()

,Country,Code,Year,business_to_personal_ratio
0,Albania,ALB,1998,0.594340
1,Albania,ALB,1999,0.982558
2,Albania,ALB,2000,0.353211
3,Albania,ALB,2001,0.399177
4,Albania,ALB,2002,0.270270


We will apply the same approach for these feature as well, but we will use a higher decay, meaning a lower com value. This will put more emphasis on more recent data.

In [17]:
gdp_tourism_proportion_adjusted = apply_ewm_and_filter(gdp_tourism_proportion, "tourism_gdp_percentage", "tourism_gdp_percentage_over_time", 0.2)
business_to_tourists_ratio_adjusted = apply_ewm_and_filter(business_to_tourists_ratio, "business_to_personal_ratio", "business_to_personal_ratio_over_time", 0.2)

In [18]:
gdp_tourism_proportion_adjusted

,Country,Code,tourism_gdp_percentage_over_time
0,Algeria,DZA,1.129921
1,Argentina,ARG,1.890788
2,Aruba,ABW,21.208039
3,Australia,AUS,1.572674
4,Australia and New Zealand,NaN,1.815653
...,...,...,...
114,United States,USA,2.875489
115,Uruguay,URY,7.188250
116,Vietnam,VNM,9.046535
117,Western Asia and Northern Africa,NaN,5.117713


In [19]:
business_to_tourists_ratio_adjusted

,Country,Code,business_to_personal_ratio_over_time
0,Albania,ALB,0.021680
1,Algeria,DZA,1.764563
2,American Samoa,ASM,0.210151
3,Andorra,AND,0.037698
4,Angola,AGO,0.111823
...,...,...,...
180,Vanuatu,VUT,0.055749
181,Venezuela,VEN,0.423833
182,Vietnam,VNM,0.201770
183,Zambia,ZMB,0.642594


## Stay of Tourists Statistics  

In [20]:
avg_len_stay.rename(columns={"Entity":"Country", "Average length of stay":"avg_stay_days"}, inplace=True)
avg_len_stay

,Country,Code,Year,avg_stay_days
0,Albania,ALB,2018,2.43
1,Albania,ALB,2019,2.58
2,Albania,ALB,2020,2.34
3,Albania,ALB,2021,2.60
4,Algeria,DZA,2000,3.02
...,...,...,...,...
2889,Zimbabwe,ZWE,2017,3.00
2890,Zimbabwe,ZWE,2018,3.00
2891,Zimbabwe,ZWE,2019,3.00
2892,Zimbabwe,ZWE,2020,3.00


In [21]:
inbound_arrivals.rename(columns={"Entity":"Country", "Inbound arrivals (tourists) per 1000 people":"inbound_arrivals"}, inplace=True)
inbound_arrivals

,Country,Code,Year,inbound_arrivals
0,Albania,ALB,2007,356.844180
1,Albania,ALB,2008,422.469850
2,Albania,ALB,2009,583.848900
3,Albania,ALB,2010,752.041750
4,Albania,ALB,2011,851.185600
...,...,...,...,...
4928,Zimbabwe,ZWE,1997,112.740204
4929,Zimbabwe,ZWE,1998,171.972400
4930,Zimbabwe,ZWE,1999,179.320500
4931,Zimbabwe,ZWE,2000,157.841200


In [22]:
domestic_tourists.rename(columns={"Entity":"Country", "Domestic trips (tourists) per 1000 people":"domestic_tourists"}, inplace=True)
domestic_tourists

,Country,Code,Year,domestic_tourists
0,Argentina,ARG,2006,524.50980
1,Argentina,ARG,2012,1262.67020
2,Argentina,ARG,2013,1266.41170
3,Argentina,ARG,2014,1173.87270
4,Argentina,ARG,2015,1051.41210
...,...,...,...,...
865,Zimbabwe,ZWE,2017,617.30990
866,Zimbabwe,ZWE,2018,584.69890
867,Zimbabwe,ZWE,2019,791.74940
868,Zimbabwe,ZWE,2020,303.58023


In [23]:
inbound_outbound_ratio.rename(columns={"Entity":"Country", "Inbound/Outbound (tourists)":"inbound_to_outbound_ratio"}, inplace=True)
inbound_outbound_ratio

,Country,Code,Year,inbound_to_outbound_ratio
0,Angola,AGO,1995,3.000000
1,Angola,AGO,1996,7.000000
2,Argentina,ARG,1995,0.600000
3,Argentina,ARG,1996,0.608473
4,Argentina,ARG,1997,0.611911
...,...,...,...,...
2105,Zimbabwe,ZWE,1995,5.324219
2106,Zimbabwe,ZWE,1996,22.855072
2107,Zimbabwe,ZWE,1997,10.414634
2108,Zimbabwe,ZWE,1998,9.323944


We can apply the same exponential weighted mean with a higher emphasis on newer data. 

In [24]:
avg_len_stay_adjusted = apply_ewm_and_filter(avg_len_stay, "avg_stay_days", "avg_stay_days_over_time", 0.2)
inbound_arrivals_adjusted = apply_ewm_and_filter(inbound_arrivals, "inbound_arrivals", "inbound_arrivals_over_time", 0.2)
domestic_tourists_adjusted = apply_ewm_and_filter(domestic_tourists, "domestic_tourists", "domestic_tourists_over_time", 0.2)
inbound_outbound_ratio_adjusted = apply_ewm_and_filter(inbound_outbound_ratio, "inbound_to_outbound_ratio", "inbound_to_outbound_ratio_over_time", 0.2)

In [25]:
avg_len_stay_adjusted

,Country,Code,avg_stay_days_over_time
0,Albania,ALB,2.562741
1,Algeria,DZA,1.621484
2,Andorra,AND,2.668918
3,Angola,AGO,4.000000
4,Argentina,ARG,2.552286
...,...,...,...
163,Vanuatu,VUT,9.294494
164,Venezuela,VEN,7.680954
165,Vietnam,VNM,6.988401
166,Zambia,ZMB,3.843949


## Putting all data together

All datasets will be combines and the "Country" and "Code" column will be taken from the food_bev_workers_adjusted dataset, having 110 unique countries. 

In [26]:
# Start with the first dataset: tourism_workers_adjusted
combined_df = tourism_workers_adjusted[['Code', 'Country', 'tourism_employment_per_1000_over_time']].copy()

# List of other datasets to merge
datasets_to_merge = [
    food_bev_workers_adjusted,
    gdp_tourism_proportion_adjusted,
    business_to_tourists_ratio_adjusted,
    avg_len_stay_adjusted,
    inbound_arrivals_adjusted,
    domestic_tourists_adjusted,
    inbound_outbound_ratio_adjusted
]

# Sequentially merge each dataset
for df in datasets_to_merge:
    # Drop 'Code' column from the current dataset before merging
    df = df.drop(columns=['Code'], errors='ignore')
    
    # Merge on 'Country', keeping 'Code' and 'Country' from combined_df
    combined_df = pd.merge(combined_df, df, on='Country', how='left')



In [27]:
combined_df

,Code,Country,tourism_employment_per_1000_over_time,food_employment_per_1000_over_time,tourism_gdp_percentage_over_time,business_to_personal_ratio_over_time,avg_stay_days_over_time,inbound_arrivals_over_time,domestic_tourists_over_time,inbound_to_outbound_ratio_over_time
0,ALB,Albania,15.608840,NaN,NaN,0.021680,2.562741,1793.802738,NaN,NaN
1,DZA,Algeria,7.411779,NaN,1.129921,1.764563,1.621484,NaN,NaN,NaN
2,AGO,Angola,2.439200,0.710283,NaN,0.111823,4.000000,2.001969,NaN,6.428571
3,ARG,Argentina,28.270541,11.444930,1.890788,0.140625,2.552286,16.460292,645.361734,0.420933
4,AUS,Australia,21.464763,7.825559,1.572674,0.280377,2.216665,247.324185,3157.386095,NaN
...,...,...,...,...,...,...,...,...,...,...
105,USA,United States,13.763542,3.456045,2.875489,0.160518,NaN,69.200533,NaN,0.612504
106,VIR,United States Virgin Islands,58.525202,25.238442,NaN,NaN,4.355942,6256.192240,NaN,NaN
107,URY,Uruguay,35.711428,14.620641,7.188250,0.084674,4.496567,913.040069,1685.925742,1.462021
108,UZB,Uzbekistan,1.545283,NaN,NaN,0.021544,2.551356,NaN,NaN,NaN


In [28]:
# Calculate the percentage of null values in each column
null_percentage = combined_df.isnull().mean() * 100

# Display the null percentage for each column
display(null_percentage)


Code                                      0.000000
Country                                   0.000000
tourism_employment_per_1000_over_time     0.000000
food_employment_per_1000_over_time       22.727273
tourism_gdp_percentage_over_time         28.181818
business_to_personal_ratio_over_time     14.545455
avg_stay_days_over_time                  15.454545
inbound_arrivals_over_time                6.363636
domestic_tourists_over_time              41.818182
inbound_to_outbound_ratio_over_time      39.090909
dtype: float64

In [29]:
combined_df.to_csv('../Data/tourism_combined_data.csv', index=False)